In [ ]:
!pip install torch

In [ ]:
import pandas as pd

df = pd.read_csv("/content/insurance.csv")
df.head()

In [ ]:
df = df.drop_duplicates()
df = df.drop(columns=["region"])
df["smoker"] = df["smoker"].map({"yes": 1, "no": 0})
df = pd.get_dummies(
    df,
    columns=["sex"],
)
df["sex_female"] = df["sex_female"].astype(int)
df["sex_male"] = df["sex_male"].astype(int)

df.head()

# Training Using Neural networks

### Train test split

In [ ]:
X = df.drop(columns=["charges"])
y = df["charges"]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

### Feature scaling

- Neural Networks are trained using gradient descent, which is sensitive to feature scale.
- If inputs have very different ranges, large-valued features dominate the learning process and slow or destabilize training.
- StandardScaler puts all features on a similar scale (mean 0, std 1), leading to faster and more stable convergence.
- Classical linear regression can often work without scaling because it has a closed-form solution and is less gradient-sensitive.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
X_train

### Convert to pytorch tensors

In [ ]:
import torch

In [ ]:
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test  = torch.tensor(X_test, dtype=torch.float32)

y_train = torch.tensor(y_train.to_numpy().reshape(-1, 1), dtype=torch.float32)
y_test  = torch.tensor(y_test.to_numpy().reshape(-1, 1), dtype=torch.float32)

### Design Your neural networks

This defines a feedforward neural network for regression using PyTorch.

The model takes 6 input features and passes them through multiple fully connected (Linear) layers with ReLU activations in between to learn non-linear relationships.

Each hidden layer gradually reduces dimensionality (32 → 16 → 8), helping the network extract useful patterns.

The final layer outputs a single value, representing the predicted insurance charges.

In [ ]:
import torch.nn as nn


class InsuranceNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(6, 32),   # 6 input features
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1)     # output: charges
        )

    def forward(self, x):
        return self.net(x)

### Create model instence, and define criterian and optimizer

- `criterion` (loss function) measures how wrong the model’s predictions are.

MSELoss computes the mean squared error, which is standard for regression problems.

- `optimizer` updates the model’s weights to reduce the loss.

Adam is an adaptive gradient descent algorithm that adjusts learning rates automatically, leading to faster and more stable training.

In [ ]:
from torch.nn import MSELoss
from torch.optim import Adam

model = InsuranceNN()
criterion = MSELoss()
optimizer = Adam(model.parameters(), lr=0.01)

## Optionally you can move your model to GPU if available

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

In [ ]:
model.to(device)

X_train = X_train.to(device)
y_train = y_train.to(device)

### Train Our model

In [ ]:
epochs = 1000

for epoch in range(epochs):
    model.train()

    predictions = model(X_train)
    loss = criterion(predictions, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 50 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.2f}")

### Evaluate our model

In [ ]:
from sklearn.metrics import r2_score
model.eval()

model = model.cpu()


with torch.no_grad():
    y_pred = model(X_test).numpy()
    y_true = y_test.numpy()

r2 = r2_score(y_true, y_pred)
print("R2 score:", r2)

In [ ]:
def predict_insurance(age, bmi, children, smoker, sex):
    new_df = pd.DataFrame(
        [{
            "age": age,
            "bmi": bmi,
            "children": children,
            "smoker": 1 if smoker else 0,
            "sex_female": 1 if sex=="female" else 0,
            "sex_male": 1 if sex=="male" else 0
        }]
    )


    new_scaled = scaler.transform(new_df)
    model.eval()

    with torch.inference_mode():
        new_tensor = torch.tensor(new_scaled, dtype=torch.float32)
        prediction = model(new_tensor)
    result = float(prediction[0][0])
    return round(result, 2)

In [ ]:
predict_insurance(
    age=26,
    bmi=27.9,
    children=3,
    smoker=True,
    sex="male"
)